In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install parler-tts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.7/100.7 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 68.7 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6

In [1]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer, AutoFeatureExtractor
import numpy as np
import nltk

ModuleNotFoundError: No module named 'parler_tts'

In [3]:
import parler_tts
print(parler_tts.__version__)

2025-05-17 04:09:50.364958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747454990.575842      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747454990.637209      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


0.2.3


In [15]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [16]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

In [17]:
torch_dtype = torch.bfloat16 if device != "cpu" else torch.float32
repo_id = "ai4bharat/indic-parler-tts"

In [18]:
model = ParlerTTSForConditionalGeneration.from_pretrained(
    repo_id, attn_implementation="eager" ,torch_dtype=torch_dtype,
).to(device);

In [19]:
tokenizer = AutoTokenizer.from_pretrained(repo_id)
description_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
feature_extractor = AutoFeatureExtractor.from_pretrained(repo_id)
SAMPLE_RATE = feature_extractor.sampling_rate
SEED = 42;

In [20]:
compile_mode = "reduce-overhead" # chose "reduce-overhead" for 3 to 4x speed-up
model.forward = torch.compile(model.forward, mode=compile_mode)

In [21]:
@torch.no_grad()
def generate(text, description):
    """
    Generate audio from text in a single pass (non-streaming)
    
    Args:
        text (str): The text to convert to speech
        description (str): Description of the voice style
        
    Returns:
        tuple: (sample_rate, audio_data)
    """
    model.generation_config.cache_implementation = "static"
    max_length = 50
    print(f"Using device: {device}")
            
    # Tokenize input text and description
    print(f"Tokenizing input text: \"{text}\"")
    inputs = description_tokenizer(description, return_tensors="pt", padding="max_length", max_length=max_length).to(device)
    # prompt = generate.tokenizer(text, return_tensors="pt").to(device)

    sentences_text = nltk.sent_tokenize(text)
    curr_sentence = ""
    chunks = []

    for sentence in sentences_text:
        candidate = " ".join([curr_sentence, sentence])
        if len(tokenizer.encode(candidate)) > 500:
            chunks.append(curr_sentence)
            curr_sentence = sentence
        else:
            curr_sentence = candidate       

    if curr_sentence != "":
        chunks.append(curr_sentence)
  
    print(chunks)

    all_audio = []
    

    for chunk in chunks:
        # Setup generation parameters
        # Batch everything in a single run for better efficiency
        prompt = tokenizer(chunk, return_tensors="pt", padding="max_length", max_length=max_length).to(device)
        generation_kwargs = dict(
            input_ids=inputs.input_ids,
            prompt_input_ids=prompt.input_ids,
            attention_mask=inputs.attention_mask,
            prompt_attention_mask=prompt.attention_mask,
            do_sample=True,
            return_dict_in_generate=True
        )

        n_steps = 1 if compile_mode == "default" else 2
        for _ in range(n_steps):
          _ = model.generate(**generation_kwargs)


        # Extract audio from generation
        if hasattr(generation, 'sequences') and hasattr(generation, 'audios_length'):
            audio = generation.sequences[0, :generation.audios_length[0]]
            audio_np = audio.to(torch.float32).cpu().numpy().squeeze()
            if len(audio_np.shape) > 1:
                audio_np = audio_np.flatten()
            all_audio.append(audio_np)
    
    # Combine all audio chunks
    combined_audio = np.concatenate(all_audio)
    
    
    print(f"Generated audio with length: {len(combined_audio)} samples")
    return SAMPLE_RATE, combined_audio

In [22]:
from IPython.display import Audio

sample_rate, audio_arr = generate(
    "ᱦᱮᱸ ᱞᱟᱹᱜᱤᱫ ᱜᱮ ᱠᱚ ᱛᱟᱹᱵᱤᱱᱟ ᱥᱟᱨᱩᱞ ᱠᱟᱰ",
    "Leela speaks in a high-pitched, fast-paced, and cheerful tone, full of energy and happiness. The recording is very high quality with no background noise"
)
Audio(audio_arr, rate=sample_rate)

Using device: cuda
Tokenizing input text: "ᱦᱮᱸ ᱞᱟᱹᱜᱤᱫ ᱜᱮ ᱠᱚ ᱛᱟᱹᱵᱤᱱᱟ ᱥᱟᱨᱩᱞ ᱠᱟᱰ"
[' ᱦᱮᱸ ᱞᱟᱹᱜᱤᱫ ᱜᱮ ᱠᱚ ᱛᱟᱹᱵᱤᱱᱟ ᱥᱟᱨᱩᱞ ᱠᱟᱰ']


The 'max_batch_size' argument of StaticCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla P100-PCIE-16GB does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla P100-PCIE-16GB does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla P100-PCIE-16GB does not support bfloat16 compilation natively, skipping
  warnings.warn(
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla P100-PCIE-16GB does not support bfloat16 compilation nativ

AttributeError: 'StaticCache' object has no attribute 'max_batch_size'